In [1]:
from simtk import unit
from openmmtools import testsystems, cache
from openmmtools.mcmc import GHMCMove, MCMCSampler, MCRotationMove, BaseIntegratorMove, IntegratorMoveError
from openmmtools.states import ThermodynamicState, SamplerState, CompoundThermodynamicState
from simtk.openmm import CompoundIntegrator
from openmmtools import alchemy
from simtk import unit
from openmmtools.utils import RestorableOpenMMObject
from openmmtools.integrators import *
import numpy as np
import copy, sys
import logging
from simtk import openmm
import parmed
from openmmtools.utils import SubhookedABCMeta, Timer, RestorableOpenMMObject
from openmmtools import testsystems, alchemy
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
np.random.RandomState(seed=3134)
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO, stream=sys.stdout)

In [2]:
# Define parameters
temperature = 300 * unit.kelvin
collision_rate = 1 / unit.picoseconds
timestep = 1.0 * unit.femtoseconds
n_steps = 10

In [3]:
# import testsystem
from openmmtools import testsystems
ala = testsystems.AlanineDipeptideVacuum()

In [4]:
# create our alchemical state
factory = alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False)
alchemical_region = alchemy.AlchemicalRegion(alchemical_atoms=range(22))
alchemical_atoms = list(alchemical_region.alchemical_atoms)
alanine_alchemical_system = factory.create_alchemical_system(
    reference_system=ala.system, alchemical_regions=alchemical_region)
alchemical_state = alchemy.AlchemicalState.from_system(alanine_alchemical_system)

In [5]:
# Enslave lambda_sterics and lambda_electrostatics to a generic lambda variable.
alchemical_state.set_alchemical_variable('lambda', 0)
print(alchemical_state.lambda_electrostatics)
print(alchemical_state.lambda_sterics)

1.0
1.0


In [7]:
# Define our alchemical function and appy to state
# The functions here turn off first electrostatic and the steric interactions
# in sequence as lambda goes from 1.0 to 0.0.
f_electrostatics = '2*(lambda-0.5)*step(lambda-0.5)'
f_sterics = '2*lambda*step_hm(0.5-lambda) + step_hm(lambda-0.5)'
alchemical_state.lambda_electrostatics = alchemy.AlchemicalFunction(f_electrostatics)
alchemical_state.lambda_sterics = alchemy.AlchemicalFunction(f_sterics)

In [8]:
# Check that we can modify the lambda varible on the state
alchemical_state.set_alchemical_variable('lambda', 0.75)
#print(dir(alchemical_state))
print(alchemical_state.lambda_electrostatics)
print(alchemical_state.lambda_sterics)

0.5
1.0


In [9]:
alchemical_state.set_alchemical_variable('lambda', 1.0)
print(alchemical_state.lambda_electrostatics)
print(alchemical_state.lambda_sterics)

1.0
1.0


In [10]:
# Create our custom State objects
thermo_state = ThermodynamicState(system=alanine_alchemical_system, temperature=300 * unit.kelvin)
compound_state = CompoundThermodynamicState(thermo_state, composable_states=[alchemical_state])

sampler_state = SamplerState(positions=ala.positions)

In [16]:
compound_state.set_alchemical_variable('lambda', 0.5)
print(compound_state.lambda_electrostatics)
print(compound_state.lambda_sterics)

print(alchemical_state.lambda_electrostatics)
print(alchemical_state.lambda_sterics)

0.0
1.0
1.0
1.0


In [11]:
# Set Integrator
ncmc_integrator = AlchemicalNonequilibriumLangevinIntegrator(
    splitting='H R V O V R H',
    #splitting='O { V R H R V } O',
    temperature=temperature,
    collision_rate=collision_rate,
    timestep=timestep,
    nsteps_neq=n_steps,
    measure_heat=True)
integrator = LangevinIntegrator(
    temperature=temperature, timestep=timestep, collision_rate=collision_rate, measure_heat=True)

compound_integrator = CompoundIntegrator()
compound_integrator.addIntegrator(ncmc_integrator)
compound_integrator.addIntegrator(integrator)
compound_integrator.setCurrentIntegrator(0)

In [12]:
# Grab the context and set state
context_cache = cache.global_context_cache
context, compound_integrator = context_cache.get_context(compound_state, compound_integrator)

# If we reassign velocities, we can ignore the ones in sampler_state.
sampler_state.apply_to_context(context)
context.setVelocitiesToTemperature(compound_state.temperature)

In [17]:
# Do some steps 
rot_step = int(n_steps/2)
for n in range(n_steps):
    compound_integrator.step(1)
    try:
        curr_integrator = compound_integrator.getIntegrator(compound_integrator.getCurrentIntegrator())
        step = curr_integrator.getGlobalVariableByName('step')
        alch_lambda = curr_integrator.getGlobalVariableByName('lambda')
        compound_state.set_alchemical_variable('lambda', alch_lambda)

        lambda_step = curr_integrator.getGlobalVariableByName('lambda_step')
        protocol_work = curr_integrator.getGlobalVariableByName('protocol_work')
        
        print('Step:', step, 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step, 'Sterics:', compound_state.lambda_sterics, 'Elec:', compound_state.lambda_electrostatics)
        if n == rot_step-1:
            break
    except Exception as e:
        print(e)

#print(dir(compound_state))


Step: 1.0 Work: 0.0 Lambda:  0.1 Lambda Step:  2.0 Sterics: 0.2 Elec: -0.0
Step: 2.0 Work: 0.0 Lambda:  0.2 Lambda Step:  4.0 Sterics: 0.4 Elec: -0.0
Step: 3.0 Work: 0.0 Lambda:  0.3 Lambda Step:  6.0 Sterics: 0.6 Elec: -0.0
Step: 4.0 Work: 0.0 Lambda:  0.4 Lambda Step:  8.0 Sterics: 0.8 Elec: -0.0
Step: 5.0 Work: 0.0 Lambda:  0.5 Lambda Step:  10.0 Sterics: 1.0 Elec: 0.0


In [15]:
# APPLY SOME PERTURBATION HERE

In [18]:
# Run our integrator to the end
rot_step = int(n_steps/2)
for n in range(n_steps):
    compound_integrator.step(1)
    try:
        curr_integrator = compound_integrator.getIntegrator(compound_integrator.getCurrentIntegrator())
        alch_lambda = curr_integrator.getGlobalVariableByName('lambda')
        compound_state.set_alchemical_variable('lambda', alch_lambda)
        step = curr_integrator.getGlobalVariableByName('step')
        lambda_step = curr_integrator.getGlobalVariableByName('lambda_step')
        protocol_work = curr_integrator.getGlobalVariableByName('protocol_work')
        print('Step:', step, 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step, 'Sterics:', compound_state.lambda_sterics, 'Elec:', compound_state.lambda_electrostatics)
        if n == rot_step-1:
            break
    except Exception as e:
        print(e)

Step: 6.0 Work: 0.0 Lambda:  0.6 Lambda Step:  12.0 Sterics: 1.0 Elec: 0.19999999999999996
Step: 7.0 Work: 0.0 Lambda:  0.7 Lambda Step:  14.0 Sterics: 1.0 Elec: 0.3999999999999999
Step: 8.0 Work: 0.0 Lambda:  0.8 Lambda Step:  16.0 Sterics: 1.0 Elec: 0.6000000000000001
Step: 9.0 Work: 0.0 Lambda:  0.9 Lambda Step:  18.0 Sterics: 1.0 Elec: 0.8
Step: 10.0 Work: 0.0 Lambda:  1.0 Lambda Step:  20.0 Sterics: 1.0 Elec: 1.0


In [19]:
# Reset the variables
curr_integrator.setGlobalVariableByName('step', 0)
curr_integrator.setGlobalVariableByName('lambda_step', 0)
curr_integrator.setGlobalVariableByName('protocol_work', 0)

In [20]:
# We get also velocities here even if we don't need them because we
# will recycle this State to update the sampler state object. This
# way we won't need a second call to Context.getState().
context_state = context.getState(
    getPositions=True, getVelocities=True, getEnergy=True, enforcePeriodicBox=compound_state.is_periodic)

# This is an optimization around the fact that Collective Variables are not a part of the State,
# but are a part of the Context. We do this call twice to minimize duplicating information fetched from
# the State.
# Update everything but the collective variables from the State object
sampler_state.update_from_context(context_state, ignore_collective_variables=True)
# Update only the collective variables from the Context
sampler_state.update_from_context(
    context, ignore_positions=True, ignore_velocities=True, ignore_collective_variables=False)

In [21]:
context, compound_integrator1 = context_cache.get_context(compound_state)
sampler_state.apply_to_context(context)
context.setVelocitiesToTemperature(compound_state.temperature)

In [22]:
# We get also velocities here even if we don't need them because we
# will recycle this State to update the sampler state object. This
# way we won't need a second call to Context.getState().
context_state = context.getState(
    getPositions=True, getVelocities=True, getEnergy=True, enforcePeriodicBox=compound_state.is_periodic)

# This is an optimization around the fact that Collective Variables are not a part of the State,
# but are a part of the Context. We do this call twice to minimize duplicating information fetched from
# the State.
# Update everything but the collective variables from the State object
sampler_state.update_from_context(context_state, ignore_collective_variables=True)
# Update only the collective variables from the Context
sampler_state.update_from_context(
    context, ignore_positions=True, ignore_velocities=True, ignore_collective_variables=False)

In [23]:
# Run our integrator to the end
rot_step = int(n_steps/2)
for n in range(n_steps):
    compound_integrator.step(1)
    try:
        curr_integrator = compound_integrator.getIntegrator(compound_integrator.getCurrentIntegrator())
        alch_lambda = curr_integrator.getGlobalVariableByName('lambda')
        compound_state.set_alchemical_variable('lambda', alch_lambda)

        step = curr_integrator.getGlobalVariableByName('step')
        lambda_step = curr_integrator.getGlobalVariableByName('lambda_step')
        protocol_work = curr_integrator.getGlobalVariableByName('protocol_work')
        print('Step:', step, 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step, 'Sterics:', compound_state.lambda_sterics, 'Elec:', compound_state.lambda_electrostatics)
        if n == rot_step-1:
            break
    except Exception as e:
        print(e)

Step: 1.0 Work: 0.0 Lambda:  0.1 Lambda Step:  2.0 Sterics: 0.2 Elec: -0.0
Step: 2.0 Work: 0.0 Lambda:  0.2 Lambda Step:  4.0 Sterics: 0.4 Elec: -0.0
Step: 3.0 Work: 0.0 Lambda:  0.3 Lambda Step:  6.0 Sterics: 0.6 Elec: -0.0
Step: 4.0 Work: 0.0 Lambda:  0.4 Lambda Step:  8.0 Sterics: 0.8 Elec: -0.0
Step: 5.0 Work: 0.0 Lambda:  0.5 Lambda Step:  10.0 Sterics: 1.0 Elec: 0.0


In [24]:
# Run our integrator to the end
rot_step = int(n_steps/2)
for n in range(n_steps):
    compound_integrator.step(1)
    try:
        curr_integrator = compound_integrator.getIntegrator(compound_integrator.getCurrentIntegrator())
        alch_lambda = curr_integrator.getGlobalVariableByName('lambda')
        compound_state.set_alchemical_variable('lambda', alch_lambda)

        step = curr_integrator.getGlobalVariableByName('step')
        lambda_step = curr_integrator.getGlobalVariableByName('lambda_step')
        protocol_work = curr_integrator.getGlobalVariableByName('protocol_work')
        print('Step:', step, 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step, 'Sterics:', compound_state.lambda_sterics, 'Elec:', compound_state.lambda_electrostatics)
        if n == rot_step-1:
            break
    except Exception as e:
        print(e)

Step: 6.0 Work: 0.0 Lambda:  0.6 Lambda Step:  12.0 Sterics: 1.0 Elec: 0.19999999999999996
Step: 7.0 Work: 0.0 Lambda:  0.7 Lambda Step:  14.0 Sterics: 1.0 Elec: 0.3999999999999999
Step: 8.0 Work: 0.0 Lambda:  0.8 Lambda Step:  16.0 Sterics: 1.0 Elec: 0.6000000000000001
Step: 9.0 Work: 0.0 Lambda:  0.9 Lambda Step:  18.0 Sterics: 1.0 Elec: 0.8
Step: 10.0 Work: 0.0 Lambda:  1.0 Lambda Step:  20.0 Sterics: 1.0 Elec: 1.0
